In [1]:
import numpy as np
import sys
sys.path.append( '../vqe-in-dft' )
from tqdm import tqdm

import vqe_in_dft 

import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci, lo, ao2mo

In [2]:
geometry = [
['H', (0.7493682,0.0000000,0.4424329)],
['O', (0.0000000,0.0000000,-0.1653507)],
    
['H', (-0.7493682,0.0000000,0.4424329)]
]

# geometry = [
# ['C', (0.0000, 0.0000, 0.0000)],
# ['H', (0.5288, 0.1610, 0.9359)],
# ['H', (0.2051, 0.8240, -0.6786)],
# ['H', (0.3345, -0.9314, -0.4496)],
# ['H', (-1.0685, -0.0537, 0.1921)]
# ]

# geometry = [
# ['H', (1.2473876659, -0.8998737590, 0.6150681570)],
# ['O', (1.2322305822, -0.2731895077, -0.1276123902)],

# ['C', (0.0849758188, 0.5590385475, 0.0510545434)],
# ['H', (0.1506137362, 1.1200249874, 0.9943015309)],
# ['H', (0.1316093068, 1.2841805400, -0.7645223601)],

# ['C', (-1.2129704155, -0.2295285634, -0.0097156258)],
# ['H', (-2.0801425360, 0.4329727646,0.0722817289)],
# ['H', (-1.2655910941, -0.9539857247, 0.8097953440)],
# ['H', (-1.2737541560, -0.7748626513, -0.9540587845)],
# ]

N_active_atoms = 2


low_level_scf_method='RKS'
high_level_scf_method='CCSD'
E_convergence_tol = 1e-6
basis = 'STO-3G' #'augccpvdz' # '6-31g' #'STO-3G'
unit= 'angstrom'
pyscf_print_level=1
memory=8000
charge=0
spin=0
run_fci= True#True
low_level_xc_functional = 'lda, vwn' # 'b3lyp'
high_level_xc_functional = 'b3lyp'

phys_notation = True

# Build the GLOBAL (full) molecule


and run supersystem calculation (cheap method)

https://pyscf.org/_modules/pyscf/gto/basis.html

In [3]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       unit=unit,
                       spin=spin,
                      )
full_system_mol.build()
full_system_mol.atom

[['H', (0.7493682, 0.0, 0.4424329)],
 ['O', (0.0, 0.0, -0.1653507)],
 ['H', (-0.7493682, 0.0, 0.4424329)]]

In [4]:
## FCI comparison
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= memory
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -75.015352478525


In [5]:
# HF_scf = scf.RHF(full_system_mol)
# HF_scf.verbose= 1
# HF_scf.max_memory= memory
# HF_scf.conv_tol = 1e-6
# HF_scf.kernel()

# my_fci = ci.CISD(HF_scf).run()

In [6]:
# Draw molecule

xyz_string = vqe_in_dft.Get_xyz_string(full_system_mol)
vqe_in_dft.Draw_molecule(xyz_string, width=400, height=400, jupyter_notebook=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
# Run global calculation

full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose= pyscf_print_level
full_system_scf.max_memory= memory
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional
full_system_scf.kernel()

-74.73474041595246

# Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}= \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

# Choose active and environment systems

## METHOD 1 

- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**

    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment
 


To choose the active and enviroment subsystems we do the following:

1. Given a localized molecular orbs (localized C matrix), we take the absolute mag squared of the coefficients of the active part for a given localized orb and divide by the absolute mag squared of all the coefficents of a that orb... THis will give a value of how much the active system contributes to that orb.

2. Mathematically, for orbital $j$ 
    - remember MO orbs given by columns of C matrix
    - In equation below C matrix is the LOCALIZED form!


$$ \text{threshold} =  \frac{\sum_{\mu\in \text{active AO}}^{K} |C_{\mu j}|^{2}}{\sum_{\mu =1}^{K} |C_{\mu j}|^{2}}$$

## METHOD 2 - SPADE

WRITE NOTES TODO

In [8]:
def Localize_orbitals(localization_method, PySCF_scf_obj, N_active_atoms, THRESHOLD=None, sanity_check=True):

    if PySCF_scf_obj.mo_coeff is None:
        raise ValueError('need to perform SCF calculation before localization')


    S_ovlp = PySCF_scf_obj.get_ovlp()
    AO_slice_matrix = PySCF_scf_obj.mol.aoslice_by_atom()

    occupied_orbs = PySCF_scf_obj.mo_coeff[:,PySCF_scf_obj.mo_occ>0]
    # run localization scheme
    if localization_method.lower() == 'spade':
        

        # Get active AO indices
        N_active_AO = AO_slice_matrix[N_active_atoms-1][3]  # find max AO index for active atoms (neg 1 as python indexs from 0)

        
        S_half = sp.linalg.fractional_matrix_power(S_ovlp , 0.5)
        orthogonal_orbitals = (S_half@occupied_orbs)[:N_active_AO, :] # Get rows (the active AO) of orthogonal orbs 

        # Comute singular vals
        u, singular_values, rotation_matrix = np.linalg.svd(orthogonal_orbitals, full_matrices=True)

        # find where largest step change 
        delta_s = singular_values[:-1] - singular_values[1:] # σ_i - σ_(i+1)
        print('delta_singular_vals:')
        print(delta_s, '\n')

        n_act_mos = np.argmax(delta_s)+1 # add one due to python indexing
        n_env_mos = len(singular_values) - n_act_mos

        # define active and environment orbitals from localization
        act_orbitals = occupied_orbs @ rotation_matrix.T[:, :n_act_mos]
        env_orbitals = occupied_orbs @ rotation_matrix.T[:, n_act_mos:]

        C_matrix_all_localized_orbitals = occupied_orbs @ rotation_matrix.T

        active_MO_inds  = np.arange(n_act_mos)
        enviro_MO_inds = np.arange(n_act_mos, n_act_mos+n_env_mos)

    else:
        if not isinstance(THRESHOLD, float):
            raise ValueError ('if localization method is not SPADE then a threshold parameter is requried to choose active system')


        # run localization scheme
        if localization_method.lower() == 'pipekmezey':
            ### PipekMezey
            PM = lo.PipekMezey(PySCF_scf_obj.mol, occupied_orbs)
            PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
            C_loc_occ = PM.kernel() # includes virtual orbs too!

        elif localization_method.lower() == 'boys':
            ### Boys
            boys_SCF = lo.boys.Boys(PySCF_scf_obj.mol, occupied_orbs)
            C_loc_occ  = boys_SCF.kernel()

        elif localization_method.lower() == 'ibo':
            ### intrinsic bonding orbs
            #
            iaos = lo.iao.iao(PySCF_scf_obj.mol, occupied_orbs)
            # Orthogonalize IAO
            iaos = lo.vec_lowdin(iaos, S_ovlp)
            C_loc_occ = lo.ibo.ibo(PySCF_scf_obj.mol, occupied_orbs, locmethod='IBO', iaos=iaos)#.kernel()

            # iaos = lo.iao.iao(PySCF_scf_obj.mol, PySCF_scf_obj.mo_coeff)
            # # Orthogonalize IAO
            # iaos = lo.vec_lowdin(iaos, S_ovlp)
            # C_loc = lo.ibo.ibo(PySCF_scf_obj.mol, PySCF_scf_obj.mo_coeff, locmethod='IBO', iaos=iaos)#.kernel()
            # C_loc_occ = C_loc[:,PySCF_scf_obj.mo_occ>0]
        else:
            raise ValueError(f'unknown localization method {localization_method}')
        
                        
        # find indices of AO of active atoms
        ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

        ### New method
        numerator_all = np.einsum('ij->j', (C_loc_occ[ao_active_inds, :])**2) # active AOs coeffs for a given MO j
        denominator_all = np.einsum('ij->j', C_loc_occ**2) # all AOs coeffs for a given MO j

        MO_active_percentage = numerator_all/denominator_all

        print('\n(active_AO^2)/(all_AO^2):', np.around(MO_active_percentage,4))
        print(f'threshold for active part: {THRESHOLD} \n')

        active_MO_inds = np.where(MO_active_percentage>THRESHOLD)[0]
        enviro_MO_inds = np.array([i for i in range(C_loc_occ.shape[1]) if i not in active_MO_inds]) # get all non active MOs


        # define active MO orbs and environment
        act_orbitals = C_loc_occ[:, active_MO_inds] # take MO (columns of C_matrix) that have high dependence from active AOs
        env_orbitals = C_loc_occ[:, enviro_MO_inds]
        
        C_matrix_all_localized_orbitals = C_loc_occ

        n_act_mos = len(active_MO_inds)
        n_env_mos = len(enviro_MO_inds)

    print(f'number of active MOs: {n_act_mos}')
    print(f'number of enviro MOs: {n_env_mos} \n')

    return act_orbitals, env_orbitals, C_matrix_all_localized_orbitals, active_MO_inds, enviro_MO_inds # C_active, C_enviro, C_all_localized, active_MO_inds, enviro_MO_inds

In [9]:
localization_method= 'PipekMezey' # 'ibo', 'Boys', 'PipekMezey' 'SPADE'
THRESHOLD = 0.95

(C_active, 
 C_envrio, 
 C_all_localized, 
 active_MO_inds,
 enviro_MO_inds) = Localize_orbitals(localization_method, 
                                     full_system_scf, 
                                     N_active_atoms, 
                                     THRESHOLD=THRESHOLD, 
                                     sanity_check=True)



(active_AO^2)/(all_AO^2): [1.     0.8478 0.747  0.9908 1.    ]
threshold for active part: 0.95 

number of active MOs: 3
number of enviro MOs: 2 



In [10]:
dm_loc = 2* C_all_localized@ C_all_localized.conj().T
dm_std = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

print('want these C_loc and C_opt_std to be different:', not np.allclose(C_all_localized,
                                                                         full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]))
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc))

want these C_loc and C_opt_std to be different: True
want dm_std and dm_loc to be the SAME: True


Virtual localized orbs!:
https://pyscf.org/_modules/pyscf/lo/vvo.html

In [11]:
from pyscf.lo import vvo

orbocc = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
orbvirt = full_system_scf.mo_coeff[:,full_system_scf.mo_occ<2]
C_virtual_loc = vvo.vvo(full_system_scf.mol,
                    orbocc,
                    orbvirt,
                    iaos=None, 
                    s=None,
                    verbose=None)

C_virtual_loc.shape

# could truncate here using active MO again!!!

# PUT localized virtual orbs into full C matrix!
C_all_localized_and_virt = np.hstack((C_all_localized,
                                     C_virtual_loc))

In [12]:
# PUT standard virtual orbs into full C matrix (no change from standard HF virutal orbs)!
C_all_localized_and_virt = np.hstack((C_all_localized,
                                     full_system_scf.mo_coeff[:,full_system_scf.mo_occ<2]))

In [13]:
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
test =  S_half@ C_virtual_loc
test[:,0].dot(test[:,1])


6.154243182198179e-16

# IMPORTANT orbital localization point!

When we performed orbital localization a unitary rotation was used... this left the generated density matrix (C * C) unchanged...

HOWEVER we need to also **rotate the Fock matrix** otherwise the orbital energies no longer make any sense (even though the final energy will be the same)!


In [14]:
# here we see the localized and standard density matrices are the same!
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc), '\n')


print('standard orb energies : \n')
print(np.diag(full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff))
print(full_system_scf.mo_energy)

print('\nLOCALIZED orb energies : \n')
print(np.diag(C_all_localized_and_virt.conj().T @ full_system_scf.get_fock() @ C_all_localized_and_virt))
print(full_system_scf.mo_energy)

# NOTE HOW THERE IS A DIFFERENCE WHEN LOCALIZED... To fix this we need to rotate the Fock matrix!

want dm_std and dm_loc to be the SAME: True 

standard orb energies : 

[-18.27447311  -0.83044897  -0.37573098  -0.15754498  -0.0584303
   0.31315706   0.41336181]
[-18.27449288  -0.8304562   -0.37573596  -0.15755407  -0.05844029
   0.31315299   0.41335685]

LOCALIZED orb energies : 

[-17.69057597  -0.59007439  -0.37573098  -0.98181669  -0.0584303
   0.31315706   0.41336181]
[-18.27449288  -0.8304562   -0.37573596  -0.15755407  -0.05844029
   0.31315299   0.41335685]


How to get the operator that changes the basis?

1. Put the standard (unlocalized) and localized C matrix into orthogonal basis
    - $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
    - $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

2. Define change of basis
    - $U = \sum_{i \in occupied} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, loc}} | + \sum_{i \in virtual} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, STD}}| $ 
    - note this projects occupied standard ortho orbs onto localized orbs
    - keeps virtual orbs the same
        - if we localize the virtual orbs then we would need to use the same procedure as the occupied orbs (easy to include)

In [15]:
# note can make this much better with einsum!

S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

ortho_std = S_half@ full_system_scf.mo_coeff
ortho_loc = S_half@ C_all_localized_and_virt

# ortho_loc[:,0].dot(ortho_loc[:,2])

U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer

print(f'is U*C_ortho_loc =  C_ortho_STD: {(np.allclose(U @ ortho_loc, ortho_std))}')
print(f'is U Udag = Identity: {np.allclose(U.conj().T@U, np.eye(U.shape[0]))}')
        
# einsum version!
np.allclose(np.einsum('ik,jk->ij', ortho_std,ortho_loc),
            U)
    
    
# ind = 0
# Fock = full_system_scf.get_fock(dm=dm_std)
# print(ortho_loc[:,ind].conj().T @ (U.conj().T @ Fock @ U) @ ortho_loc[:,ind])
# print(ortho_std[:,ind].conj().T  @ Fock @ ortho_std[:,ind])

is U*C_ortho_loc =  C_ortho_STD: True
is U Udag = Identity: True


True

Note that $U$ is currently defined using ORTHOGONAL BASIS... we want it in our usual MO basis

Remembering that

$$C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$$

Clearly to get back to standard basis we do:

$$C_{\text{std}} = S^{-0.5} C_{\text{std}}^{\text{ORTHO}}$$


NOW as 


$$U C_{\text{loc}}^{\text{ORTHO}} =C_{\text{std}}^{\text{ORTHO}}$$

we can re-write this using our definitions:
- $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
- $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

$$U (S^{0.5}C_{\text{std}}) = (S^{0.5}C_{\text{loc}})$$

Now we want to get rid of $S^{0.5}$ on RHS so we times on left by $S^{-0.5}$

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = C_{\text{loc}}$$

Therefore the change of basis operator in our non-ortho basis is simply:

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = V$$

Where: $ V C_{\text{std}} = C_{\text{loc}}$


In [16]:
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

V = S_neg_half @ U @ S_half


print('V @ C_loc = C_std:', np.allclose(V@C_all_localized_and_virt,
                                            full_system_scf.mo_coeff))

V @ C_loc = C_std: True


NOW we can check orbital energies of localized system when we use V to change basis:

In [17]:
print('standard orb energies : \n')
print(np.diag(full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff))
print(full_system_scf.mo_energy)

print('\nLOCALIZED orb energies WITHOUT Fock change of basis : \n')
print(np.diag(C_all_localized.conj().T @ full_system_scf.get_fock() @ C_all_localized))
print(full_system_scf.mo_energy)


print('\nLOCALIZED orb energies WITH Fock change of basis : \n')
print(np.diag(C_all_localized_and_virt.conj().T @ (V.conj().T @ full_system_scf.get_fock() @ V) @ C_all_localized_and_virt))
print(full_system_scf.mo_energy)

standard orb energies : 

[-18.27447311  -0.83044897  -0.37573098  -0.15754498  -0.0584303
   0.31315706   0.41336181]
[-18.27449288  -0.8304562   -0.37573596  -0.15755407  -0.05844029
   0.31315299   0.41335685]

LOCALIZED orb energies WITHOUT Fock change of basis : 

[-17.69057597  -0.59007439  -0.37573098  -0.98181669  -0.0584303 ]
[-18.27449288  -0.8304562   -0.37573596  -0.15755407  -0.05844029
   0.31315299   0.41335685]

LOCALIZED orb energies WITH Fock change of basis : 

[-18.27447311  -0.83044897  -0.37573098  -0.15754498  -0.0584303
   0.31315706   0.41336181]
[-18.27449288  -0.8304562   -0.37573596  -0.15755407  -0.05844029
   0.31315299   0.41335685]


In [18]:
# modify H_core and Veff to allow this change of basis!!!

def Get_new_Hcore(H_core, Unitary_rot):
    H_core_rot = Unitary_rot.conj().T @ H_core @Unitary_rot 
    return H_core_rot

from pyscf.dft.rks import get_veff as RKS_get_veff
## RKS_get_veff gives Coulomb + XC functional
## in matrix form: Veff = J + Vxc.
    
from pyscf import lib
from pyscf.dft import numint

def Get_new_Veff(pyscf_obj, Unitary, dm=None, check_result=False):
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    
#     Evaluate RKS/UKS XC functional and potential matrix on given meshgrids
#     for a set of density matrices.
    nelec, exc, vxc = numint.nr_vxc(pyscf_obj.mol,
                                            pyscf_obj.grids,
                                            pyscf_obj.xc,
                                            density_mat)
    vxc =  Unitary.conj().T @ vxc @ Unitary
    
    Veff = RKS_get_veff(pyscf_obj, dm=density_mat)
    if Veff.vk is not None:
        K = Unitary.conj().T @ Veff.vk @ Unitary
        J = Unitary.conj().T @ Veff.vj @ Unitary
        vxc += J - K * .5
    else:
        J = Unitary.conj().T @ Veff.vj @ Unitary
        K = None
        vxc += J 
    
    if check_result is True:
        M1 = Unitary.conj().T @ Veff.__array__() @ Unitary
        if not np.allclose(vxc, M1):
            raise ValueError('Veff in new basis NOT correct')
    
    ecoul = np.einsum('ij,ji', density_mat, J).real * .5 # note J matrix is in new basis!
    ## this ecoul term changes if the full density matrix is NOT 
    # (aka for dm_active and dm_enviroment we get different V_eff under different bases!)
    
    
    output = lib.tag_array(vxc, ecoul=ecoul, exc=Veff.exc, vj=J, vk=K)
    return output


In [19]:
# Check pyscf obj with localized orbs and unitary change of basis gives same results as before!

test_full_system_scf = scf.RKS(full_system_mol)
test_full_system_scf.verbose= pyscf_print_level
test_full_system_scf.max_memory= memory
test_full_system_scf.conv_tol = 1e-6
test_full_system_scf.xc = low_level_xc_functional


Hcore_std = test_full_system_scf.get_hcore()
test_full_system_scf.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


test_full_system_scf.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    test_full_system_scf,
                                                                                       V,
                                                                                        dm=dm)


test_full_system_scf.mo_coeff = C_all_localized_and_virt
test_full_system_scf.mo_occ = full_system_scf.mo_occ

test_full_system_scf.energy_tot(dm=dm_loc)

-74.7347404160195

In [20]:
C_std = full_system_scf.mo_coeff

print(np.allclose(C_std.conj().T @ full_system_scf.get_hcore() @ C_std,
           C_all_localized_and_virt.conj().T @ test_full_system_scf.get_hcore() @ C_all_localized_and_virt))

print(np.allclose(C_std.conj().T @ full_system_scf.get_veff(dm=dm_std) @ C_std,
           C_all_localized_and_virt.conj().T @ test_full_system_scf.get_veff(dm=dm_loc) @ C_all_localized_and_virt))

True
True


In [21]:
### RKS energy (not the same as HFock!)
vhf_std = full_system_scf.get_veff(dm=dm_std)
e1_std = np.einsum('ij,ji->', full_system_scf.get_hcore(), dm_std)
e2_std = vhf_std.ecoul + vhf_std.exc
print(e1_std + e2_std)

print()
vhf_loc = test_full_system_scf.get_veff(dm=dm_std)
e1_loc = np.einsum('ij,ji->', test_full_system_scf.get_hcore(), dm_loc)
e2_loc = vhf_loc.ecoul + vhf_loc.exc
print(e1_loc+e2_loc)

-83.8630241442354

-83.86302414430241


In [22]:
print('check electronic energies match:', np.allclose(full_system_scf.energy_elec(dm=dm_std)[0],
                                test_full_system_scf.energy_elec(dm=dm_loc)[0]))

# test_full_system_scf.kernel()  # run to populate mo_energy....
# print('check MO energies match:', np.allclose(full_system_scf.mo_energy,
#                                 test_full_system_scf.mo_energy))

check electronic energies match: True


In [23]:
F_new = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm_loc) # Note this is now in new basis!
print(np.around(C_all_localized_and_virt.conj().T @ F_new @ C_all_localized_and_virt, 2))


F_std = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_loc) # Note this is standard HF!
C_std = full_system_scf.mo_coeff
print(np.around(C_std.conj().T @ F_std @ C_std, 2))

[[-18.27  -0.     0.     0.     0.    -0.    -0.  ]
 [ -0.    -0.83   0.    -0.     0.     0.    -0.  ]
 [  0.     0.    -0.38   0.     0.    -0.     0.  ]
 [  0.    -0.     0.    -0.16  -0.     0.     0.  ]
 [  0.     0.     0.    -0.    -0.06   0.    -0.  ]
 [ -0.     0.    -0.     0.     0.     0.31   0.  ]
 [ -0.    -0.     0.     0.    -0.     0.     0.41]]
[[-18.27  -0.     0.     0.     0.    -0.     0.  ]
 [ -0.    -0.83   0.    -0.     0.     0.    -0.  ]
 [  0.     0.    -0.38   0.    -0.    -0.     0.  ]
 [  0.    -0.     0.    -0.16  -0.     0.    -0.  ]
 [  0.     0.    -0.    -0.    -0.06   0.    -0.  ]
 [ -0.     0.    -0.     0.     0.     0.31   0.  ]
 [  0.    -0.     0.    -0.    -0.     0.     0.41]]


Basically we now have orbital energies matching as we want

(This will become important when we move to post Hartree-Fock methods!)

In [24]:
# # Draw active orbitals

# list_active_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              active_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

# list_enviro_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              enviro_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [25]:
# list_active_orbitals[0]

In [26]:
# list_enviro_orbitals[0]

In [27]:
def Get_active_and_envrio_dm(PySCF_scf_obj, C_active, C_envrio, C_all_localized, sanity_check=True):

    # get density matrices
    dm_active =  2 * C_active @ C_active.T
    dm_enviro =  2 * C_envrio @ C_envrio.T

    if sanity_check:

        S_ovlp = PySCF_scf_obj.get_ovlp()

        ## check number of electrons is still the same after orbitals have been localized (change of basis)
        N_active_electrons = np.trace(dm_active@S_ovlp)
        N_enviro_electrons = np.trace(dm_enviro@S_ovlp)
        N_all_electrons = PySCF_scf_obj.mol.nelectron

        bool_flag_electron_number = np.isclose(( N_active_electrons + N_enviro_electrons), N_all_electrons)
        if not bool_flag_electron_number:
            raise ValueError('number of electrons in localized orbitals is incorrect')
        print(f'N_active_elec + N_environment_elec = N_total_elec is: {bool_flag_electron_number}')

        # checking denisty matrix parition makes sense:
        dm_localised_full_system = 2* C_all_localized@ C_all_localized.conj().T
        bool_density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
        if not bool_density_flag:
            raise ValueError('gamma_full != gamma_active + gamma_enviro')
        print(f'y_active + y_enviro = y_total is: {bool_density_flag}')

    return dm_active, dm_enviro

In [28]:
dm_active, dm_enviro = Get_active_and_envrio_dm(
                                            full_system_scf,
                                            C_active, 
                                            C_envrio, 
                                            C_all_localized, 
                                            sanity_check=True)

N_active_elec + N_environment_elec = N_total_elec is: True
y_active + y_enviro = y_total is: True


In [29]:
two_e_term =  test_full_system_scf.get_veff(dm=dm_loc)

np.allclose(V.conj().T @test_full_system_scf.get_j(dm = dm_loc) @ V,
            two_e_term.vj)

True

In [30]:
def Get_energy_and_matrices_from_dm(PySCF_scf_obj, dm_matrix, check_E_with_pyscf=True):
    """
    Get Energy from denisty matrix

    Note this uses the standard hcore (NO embedding potential here!)
    """

    # It seems that PySCF lumps J and K in the J array 
    two_e_term =  PySCF_scf_obj.get_veff(dm=dm_matrix)
    J_mat = two_e_term.vj
    K_mat = np.zeros_like(J_mat)
    
    e_xc = two_e_term.exc
    v_xc = two_e_term - J_mat 

#     H_core_standard = scf.hf.get_hcore(PySCF_scf_obj.mol) # No embedding potential
    Energy_elec = (np.einsum('ij,ji->', PySCF_scf_obj.get_hcore(), dm_matrix) + 
                   two_e_term.ecoul + two_e_term.exc)
    
    if check_E_with_pyscf:
        Energy_elec_pyscf = PySCF_scf_obj.energy_elec(dm=dm_matrix)[0]
        if not np.isclose(Energy_elec_pyscf, Energy_elec):
            raise ValueError('Energy calculation incorrect')

    return Energy_elec, J_mat, K_mat, e_xc, v_xc

In [31]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( full_system_scf, # <-- standard
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)
print(full_system_scf.energy_elec(dm=dm_active)[0])
E_act

-77.04777624866014


-77.04777624866014

In [32]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( test_full_system_scf, # <-- modified 
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)
print(test_full_system_scf.energy_elec(dm=dm_active)[0])
E_act

-76.1779206784814


-76.17792067848141

In [33]:
## Note different energies!
print(full_system_scf.energy_elec(dm=dm_active)[0])
print(test_full_system_scf.energy_elec(dm=dm_active)[0])

## This is expected as now using active density matrix (rather than full one)

print()
# If we use the full dm matrix then we get matching results (aka now have full density matrix)
E_loc = test_full_system_scf.energy_elec(dm=dm_active+dm_enviro)[0]
E_std = full_system_scf.energy_elec(dm=dm_active+dm_enviro)[0] 
print(E_loc)
print(E_std)

-77.04777624866014
-76.1779206784814

-83.86302414430244
-83.86302414423538


In [34]:
# Note we want to use modified object, where change of basis on Fock matrix has been performed!

In [35]:
# use enviro density
E_env, J_env, K_env, e_xc_env, v_xc_env = Get_energy_and_matrices_from_dm(
                                                                            test_full_system_scf, # <-- modified 
                                                                             dm_enviro, # <- ENVIRO
                                                                             check_E_with_pyscf=True)
print(test_full_system_scf.energy_elec(dm=dm_enviro)[0])
E_env

-24.360503838432965


-24.360503838432965

In [36]:
def Get_cross_terms(PySCF_scf_obj, dm_active, dm_enviro, J_env, J_act, e_xc_act, e_xc_env):
    """
    Get cross system terms
    
    """

    two_e_term_total =  PySCF_scf_obj.get_veff(dm=dm_active+dm_enviro)
    e_xc_total = two_e_term_total.exc

    j_cross = 0.5 * ( np.einsum('ij,ij', dm_active, J_env) + np.einsum('ij,ij', dm_enviro, J_act) )
    k_cross = 0.0

    xc_cross = e_xc_total - e_xc_act - e_xc_env
    two_e_cross = j_cross + k_cross + xc_cross
    
    return two_e_cross

In [37]:
# cross terms!
two_e_cross = Get_cross_terms(test_full_system_scf, # <-- modified 
                                         dm_active, 
                                         dm_enviro, 
                                         J_env, 
                                         J_act, 
                                         e_xc_act,
                                         e_xc_env)

In [38]:
print(f'E_active: {E_act}')
print(f'E_enviro: {E_env}')
print(f'E_cross: {two_e_cross}')

E_active: -76.17792067848141
E_enviro: -24.360503838432965
E_cross: 16.675400372611943


In [39]:
print(f'E_hf_standard : {full_system_scf.e_tot}')
print(f'E_act + E_env + non_add_two_e + E_nuc : {E_act+E_env+two_e_cross+ full_system_scf.energy_nuc()}')

# equation 1 of SPADE paper
print(f'energies match {np.isclose(full_system_scf.e_tot, (E_act+E_env+two_e_cross+ full_system_scf.energy_nuc()))}') # equation 3 of SPADE paper
full_system_scf.e_tot - (E_act+E_env+two_e_cross+ full_system_scf.energy_nuc())

E_hf_standard : -74.73474041595246
E_act + E_env + non_add_two_e + E_nuc : -74.73474041601949
energies match True


6.703260169160785e-11

In [40]:
print(np.trace(S_mat@dm_enviro))
print(np.trace(S_mat@dm_active))

3.9999999999999956
6.000000000000001


# NEXT define the projector onto subsystem B (enviroment)

$$P^{env} = \sum_{i \in \text{env}} |\psi_{i} \rangle \langle \psi_{i}|$$

important points:
1. Currently C_localized matrix columns give MO orbs $|\psi_{i} \rangle$
2. However, these are **not orthogonal**! Therefore defining a projector here is HARD
3. To fix this:
    - change to orthogonal basis
    - now define projector in this new basis
    - undo basis change to give projector in non-ortho basis!

In [530]:
## manual projector

## 1. convert to orthogonal C_matrix
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt # orthogonal C matrix (localized)

## 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
##### (do this in orthongoal C_matrix!)
### not we only take MO environment indices!
PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue

print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, enviro_MO_inds], 
        Loc_Ortho[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Is subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!
        
        
#### 3. NEXT USING PROJECTOR in ortho basis define projector via different methods
        
        
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
    Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    
    # Huzinaga
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    
    projector = S_half @ projector_ortho  @ S_half
    
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector_ortho = mu * PROJ_ortho
    
#     projector = S_neg_half @ projector_ortho  @ S_half

    # mu shift
    projector = S_half @ projector_ortho  @ S_half
else:
    raise ValueError(f'Unknown projection method {projector_method}')

##### 4. Define projector in standard (non-orthogonal basis)
# projector = S_half @ projector_ortho  @ S_half


### 5. now can build relevant ops



### TODO check this!!!
# when using full_system_scf and mu_shift get correct result
# but I think one should use test_full_system_scf (with modified Fock matrix) 

### NOTE: we do NOT use modified Fock matrix here!!!!!
# g_A_and_B = full_system_scf.get_veff(dm=dm_active+dm_enviro)
# g_A = full_system_scf.get_veff(dm=dm_active)

# modified Fock matrix gives correct result when change of basis used on embedded pyscf obj!!!
g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
g_A = test_full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector

Are subsystem B (env) projected onto themselves: True
Is subsystem A traced out?: True


In [42]:
# old mu shift
# projector_mu = mu * (S_mat @ dm_enviro  @ S_mat)
# np.allclose(projector_mu, projector_mu/2)

In [531]:
Fock_loc_basis = test_full_system_scf.get_fock()

if projector_method == 'mu_shfit':
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)

    ## expect subsystem A orbital energies to remain the SAME
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                        full_system_scf.mo_energy[active_MO_inds])} \n''')

    # expect subsystem B orbital energies to be pushed up to high energy
    print(f'original MO energies env: {full_system_scf.mo_energy[enviro_MO_inds]}')
    print(f'new MO energies env: {new_orbital_energies[enviro_MO_inds]}')
    # print(f'''enviro subsystem orbital energies energy increased by mu: 
    #                                                     {np.allclose(new_orbital_energies[enviro_MO_inds],
    #                                                     mu*full_system_scf.mo_energy[enviro_MO_inds])}''')
    
    
elif projector_method == 'huzinaga':
    ## projector should give opposite sign val of subsytem B orbitals vs standard Fock eval
    proj_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @projector@C_all_localized_and_virt[:, enviro_MO_inds]
    fock_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @Fock_loc_basis@C_all_localized_and_virt[:, enviro_MO_inds]
    
    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                    full_system_scf.mo_energy[active_MO_inds])} \n''')
    
    print(f'''enviro subsystem orbital energies should be ZERO (traced out):{ 
                                   np.allclose(new_orbital_energies[enviro_MO_inds],
                                                np.zeros_like(new_orbital_energies[enviro_MO_inds]))} \n''')
    
else:
    raise ValueError(f'Unknown projection method {projector_method}')


print('\nbasically want active MOs first... then enviroment')
print(f'active MO indices: {active_MO_inds} \n')
print('should appear first when orbital inds listed by energy:')
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

 <env| P |env> = - <env| F |env> : True 

active subsystem orbital energies the same: False 

enviro subsystem orbital energies should be ZERO (traced out):True 


basically want active MOs first... then enviroment
active MO indices: [0 3 4] 

should appear first when orbital inds listed by energy:


[(0, -18.2744731084804),
 (3, -0.15754497854277807),
 (4, -0.05843029691852217),
 (2, -2.748817690493042e-15),
 (1, 9.226058215349417e-16),
 (5, 0.3131570593453421),
 (6, 0.41336180729492533)]

In [44]:
# TODO ^^^^ choose frozen orbitals here!

In [45]:
print('Trace(y_ENV @ PB):', np.einsum('ij, ij', dm_enviro, projector) )
print('Trace(y_act @ PB):', np.einsum('ij, ij', dm_active, projector) )
print()
print('Trace(y_ENV @ v_emb):', np.einsum('ij, ij', dm_enviro, v_emb) )
print('Trace(y_act @ v_emb):', np.einsum('ij, ij', dm_active, v_emb) )

Trace(y_ENV @ PB): 2.412359909725771
Trace(y_act @ PB): 1.5959455978986625e-16

Trace(y_ENV @ v_emb): 12.00284837398232
Trace(y_act @ v_emb): 16.212733985519854


In [46]:
# full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
#                       basis=basis,
#                        charge=charge,
#                        spin=spin,
#                       )
# full_system_mol_EMBEDDED.build()

# # RE-DEFINE number of electrons in system
# full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

# EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
# EMBEDDED_full_system_scf.verbose=1
# EMBEDDED_full_system_scf.max_memory= memory
# EMBEDDED_full_system_scf.conv_tol = 1e-6
# EMBEDDED_full_system_scf.xc = low_level_xc_functional

# h_core = EMBEDDED_full_system_scf.get_hcore()

# # overwrite h_core to include embedding term!!!!
# EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + h_core

# E_emb = EMBEDDED_full_system_scf.kernel()

# print(f'embedded Energy: {E_emb}')



full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional



## include change of basis!
Hcore_std = EMBEDDED_full_system_scf.get_hcore()
EMBEDDED_full_system_scf.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


EMBEDDED_full_system_scf.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    EMBEDDED_full_system_scf,
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + Hcore_rotated

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

embedded Energy: -50.83689240646143


In [47]:
print(f'''number of electrons is correct: {
                                            np.isclose(sum(EMBEDDED_full_system_scf.mo_occ),
                                            2*len(active_MO_inds))
                                          }''')

number of electrons is correct: True


In [48]:
C_active_EMBEDDED = EMBEDDED_full_system_scf.mo_coeff[:, EMBEDDED_full_system_scf.mo_occ>0]
dm_active_EMBEDDED = 2* C_active_EMBEDDED @ C_active_EMBEDDED.conj().T

# energy with STANDARD rks object!
print(test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED)[0])

-76.17790569776247


In [49]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', v_emb, dm_active_EMBEDDED-dm_active)

e_act_emb = test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED,
                                        vhf= test_full_system_scf.get_veff(dm=dm_active_EMBEDDED),
                                       h1e = test_full_system_scf.get_hcore())[0]

# e_act_emb = full_system_scf.energy_elec(dm=dm_active_EMBEDDED,
#                                         vhf= full_system_scf.get_veff(dm=dm_active_EMBEDDED),
#                                        h1e = full_system_scf.get_hcore())[0]

e_mf_emb = e_act_emb + E_env + two_e_cross + test_full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

-74.7347298578023

In [50]:
full_system_scf.e_tot

-74.73474041595246

In [51]:
print(f'''global DFT calculation == seperated calculation: {
                                                            np.isclose(e_mf_emb,
                                                                        full_system_scf.e_tot)
                                                        }''')
# expected as same functional used!

global DFT calculation == seperated calculation: True


# Run more expensive DFT calculation!

In [52]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= memory
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS


## include change of basis!
Hcore_std = full_system_scf_HIGH_LEVEL.get_hcore()
full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


full_system_scf_HIGH_LEVEL.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    full_system_scf_HIGH_LEVEL, #<-need this here for more expensive xc fucnctional@
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = full_system_scf_HIGH_LEVEL.get_hcore()

# overwrite h_core to include embedding term!!!!
full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: v_emb + Hcore_rotated

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.kernel()
e_act_emb_HIGH_LVL

-51.437089306960786

In [53]:
C_active_EMBEDDED_high_lvl = full_system_scf_HIGH_LEVEL.mo_coeff[:, full_system_scf_HIGH_LEVEL.mo_occ>0]
dm_active_EMBEDDED_high_lvl = 2 * C_active_EMBEDDED_high_lvl @ C_active_EMBEDDED_high_lvl.conj().T

In [54]:
# calculate embedding correction term

dm_correction_high_lvl = np.einsum('ij, ij', v_emb, dm_active_EMBEDDED_high_lvl-dm_active)

e_act_emb_high_lvl = test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED_high_lvl,
                                        vhf= test_full_system_scf.get_veff(dm=dm_active_EMBEDDED_high_lvl),
                                       h1e = test_full_system_scf.get_hcore())[0]

E_high_lvl_DFT = e_act_emb_high_lvl + E_env + two_e_cross + test_full_system_scf.energy_nuc() + dm_correction_high_lvl
E_high_lvl_DFT # <-- energy from embedded DFT calc

-74.78363650084431

In [55]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

High level DFT in DFT error: 0.23171597768050844
LOW level DFT in DFT error: 0.2806226207225251


In [56]:
full_system_scf_HIGH_LEVEL.mo_energy

array([-1.88264774e+01, -2.22511747e-01, -1.45234285e-01, -7.54321269e-02,
       -2.44029230e-03,  2.71656171e-01,  4.45589594e-01])

In [57]:
vzcx

NameError: name 'vzcx' is not defined

# WF hartree-Fock run

array([0, 3, 4])

In [ ]:
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                    full_system_scf.mo_energy[active_MO_inds])} \n''')

In [569]:
def Get_new_Veff_HFock(pyscf_obj, Unitary, dm=None, check_result=False, hermi=1):
    
    # note this is NOT the same as RKS defintion!
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    vj, vk = pyscf_obj.get_jk(dm=density_mat, hermi=hermi)
    Veff = vj - vk * .5
    
#     Veff = pyscf_obj.get_veff(dm=density_mat)
    Veff_new = Unitary.conj().T @ Veff @ Unitary

    return Veff_new

from pyscf import lib
from pyscf.lib import logger
import numpy

from pyscf import __config__

WITH_META_LOWDIN = getattr(__config__, 'scf_analyze_with_meta_lowdin', True)
PRE_ORTH_METHOD = getattr(__config__, 'scf_analyze_pre_orth_method', 'ANO')
MO_BASE = getattr(__config__, 'MO_BASE', 1)
TIGHT_GRAD_CONV_TOL = getattr(__config__, 'scf_hf_kernel_tight_grad_conv_tol', True)
MUTE_CHKFILE = getattr(__config__, 'scf_hf_SCF_mute_chkfile', False)

def kernel_NEW(mf,
               dm_constant_env,
               dm_active,
               pyscf_DFT,
               PROJ_ortho,
               conv_tol=1e-10,
               conv_tol_grad=None,
               dump_chk=True,
               dm0=None, 
               callback=None,
               conv_check=True,
               **kwargs):
    '''kernel: the SCF driver.
    Args:
        mf : an instance of SCF class
            mf object holds all parameters to control SCF.  One can modify its
            member functions to change the behavior of SCF.  The member
            functions which are called in kernel are
            | mf.get_init_guess
            | mf.get_hcore
            | mf.get_ovlp
            | mf.get_veff
            | mf.get_fock
            | mf.get_grad
            | mf.eig
            | mf.get_occ
            | mf.make_rdm1
            | mf.energy_tot
            | mf.dump_chk
    Kwargs:
        conv_tol : float
            converge threshold.
        conv_tol_grad : float
            gradients converge threshold.
        dump_chk : bool
            Whether to save SCF intermediate results in the checkpoint file
        dm0 : ndarray
            Initial guess density matrix.  If not given (the default), the kernel
            takes the density matrix generated by ``mf.get_init_guess``.
        callback : function(envs_dict) => None
            callback function takes one dict as the argument which is
            generated by the builtin function :func:`locals`, so that the
            callback function can access all local variables in the current
            envrionment.
    Returns:
        A list :   scf_conv, e_tot, mo_energy, mo_coeff, mo_occ
        scf_conv : bool
            True means SCF converged
        e_tot : float
            Hartree-Fock energy of last iteration
        mo_energy : 1D float array
            Orbital energies.  Depending the eig function provided by mf
            object, the orbital energies may NOT be sorted.
        mo_coeff : 2D array
            Orbital coefficients.
        mo_occ : 1D array
            Orbital occupancies.  The occupancies may NOT be sorted from large
            to small.
    Examples:
    >>> from pyscf import gto, scf
    >>> mol = gto.M(atom='H 0 0 0; H 0 0 1.1', basis='cc-pvdz')
    >>> conv, e, mo_e, mo, mo_occ = scf.hf.kernel(scf.hf.SCF(mol), dm0=numpy.eye(mol.nao_nr()))
    >>> print('conv = %s, E(HF) = %.12f' % (conv, e))
    conv = True, E(HF) = -1.081170784378
    '''
    if 'init_dm' in kwargs:
        raise RuntimeError('''
You see this error message because of the API updates in pyscf v0.11.
Keyword argument "init_dm" is replaced by "dm0"''')
#     cput0 = (logger.process_clock(), logger.perf_counter())
#     if conv_tol_grad is None:
#         conv_tol_grad = numpy.sqrt(conv_tol)
#         logger.info(mf, 'Set gradient conv threshold to %g', conv_tol_grad)

    mol = mf.mol
    if dm0 is None:
        h1e = mf.get_hcore()
        s1e = mf.get_ovlp()
        mo_energy, mo_coeff = mf.eig(h1e, s1e)
        mo_occ = mf.get_occ(mo_energy, mo_coeff)
        dm = mf.make_rdm1(mo_coeff, mo_occ)
    else:
        dm = dm0

    h1e = mf.get_hcore(mol)
    vhf = mf.get_veff(mol, dm)
    s1e = mf.get_ovlp(mol)
    
    S_neg_half = sp.linalg.fractional_matrix_power(s1e , -0.5)
    S_half = sp.linalg.fractional_matrix_power(s1e , 0.5)
    e_tot = mf.energy_tot(dm, h1e, vhf)
#     logger.info(mf, 'init E= %.15g', e_tot)

    scf_conv = False
    mo_energy = mo_occ = None

    
    cond = lib.cond(s1e)
    logger.debug(mf, 'cond(S) = %s', cond)
    if numpy.max(cond)*1e-17 > conv_tol:
        logger.warn(mf, 'Singularity detected in overlap matrix (condition number = %4.3g). '
                    'SCF may be inaccurate and hard to converge.', numpy.max(cond))

    # Skip SCF iterations. Compute only the total energy of the initial density
    if mf.max_cycle <= 0:
        fock = mf.get_fock(h1e, s1e, vhf, dm)  # = h1e + vhf, no DIIS
        mo_energy, mo_coeff = mf.eig(fock, s1e)
        mo_occ = mf.get_occ(mo_energy, mo_coeff)
        return scf_conv, e_tot, mo_energy, mo_coeff, mo_occ

    if isinstance(mf.diis, lib.diis.DIIS):
        mf_diis = mf.diis
    elif mf.diis:
        assert issubclass(mf.DIIS, lib.diis.DIIS)
        mf_diis = mf.DIIS(mf, mf.diis_file)
        mf_diis.space = mf.diis_space
        mf_diis.rollback = mf.diis_space_rollback
    else:
        mf_diis = None

    if dump_chk and mf.chkfile:
        # Explicit overwrite the mol object in chkfile
        # Note in pbc.scf, mf.mol == mf.cell, cell is saved under key "mol"
        chkfile.save_mol(mol, mf.chkfile)

    # A preprocessing hook before the SCF iteration
    mf.pre_kernel(locals())

#     cput1 = logger.timer(mf, 'initialize scf', *cput0)
    for cycle in range(mf.max_cycle):
        dm_last = dm
        last_hf_e = e_tot

#         print(mo_coeff.shape)
#         Loc_Ortho = S_half@ mo_coeff # orthogonal C matrix (localized)
#         PROJ_ortho = np.zeros_like(S_mat)
#         for MO_ind in range(Loc_Ortho.shape[1]):
#             if MO_ind in enviro_MO_indices:
#                 outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
#                 PROJ_ortho+=outer
        
        # add env
#         Fock_emb = mf.get_fock(h1e, s1e, vhf, dm, cycle, mf_diis) # active_emb

        # build full Fock(gammaAemb + gamma_B_env)
        Fock_emb = mf.get_fock(h1e, s1e, vhf, dm+dm_constant_env, cycle, mf_diis) # active_emb + enviro_const
        Fock_ortho = S_neg_half@Fock_emb @ S_neg_half
        # Huzinaga
        projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
        projector = S_half @ projector_ortho  @ S_half
#         print(F_orb_energy)
        
        
#         g_A_and_B = pyscf_DFT.get_veff(dm=dm+dm_enviro)
        g_A = pyscf_DFT.get_veff(dm=dm_active)
        g_A_and_B = mf.get_veff(dm=dm_active+dm_constant_env)
#         g_A_and_B = mf.get_veff(dm=dm+dm_constant_env)
#         g_A = mf.get_veff(dm=dm)
        v_emb = g_A_and_B - g_A + projector
#         v_emb = projector

        
        proj_orb_energy = np.diag(mo_coeff.conj().T @(Fock_emb+v_emb)@mo_coeff)
#         F_orb_energy = np.diag(mo_coeff.conj().T @Fock_emb@mo_coeff)
        print(proj_orb_energy)
        print()

#         new_orbital_energies = np.diag(mo_coeff.conj().T @ (Fock_emb+projector) @ mo_coeff)
#         print(new_orbital_energies)
        
#         proj_orb_energy = mo_coeff.conj().T @projector@mo_coeff
#         fock_orb_energy = mo_coeff.conj().T @Fock_emb@mo_coeff
#         print(proj_orb_energy)# + fock_orb_energy)
       
        
    
#         proj_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @projector@mo_coeff[:, enviro_MO_inds]
#         fock_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @Fock_emb@mo_coeff[:, enviro_MO_inds]
    
#         print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
#                                                                   fock_orb_energy)} \n''')
        

#         g_A_and_B = pyscf_DFT.get_veff(dm=dm_active+dm_enviro)
# #         g_A = pyscf_DFT.get_veff(dm=dm_active)
#         g_A = mf.get_veff(dm=dm_active)
#         v_emb = g_A_and_B - g_A + projector
        
        
        
        fock = mf.get_fock(h1e, s1e, vhf, dm, cycle, mf_diis)
        fock+=v_emb
    
        fock_orb_energy = mo_coeff.conj().T @fock@mo_coeff
#         print(fock_orb_energy)
        
#         proj_orb_energy = mo_coeff.conj().T @projector@mo_coeff
#         print(np.diag(proj_orb_energy))
#         print(np.diag(fock_orb_energy))
#         print()
#         fock_orb_energy = mo_coeff.conj().T @fock@mo_coeff
#         print(np.where(-proj_orb_energy == fock_orb_energy))
        
        mo_energy, mo_coeff = mf.eig(fock, s1e)
        mo_occ = mf.get_occ(mo_energy, mo_coeff)
        dm = mf.make_rdm1(mo_coeff, mo_occ)
        # attach mo_coeff and mo_occ to dm to improve DFT get_veff efficiency
        dm = lib.tag_array(dm, mo_coeff=mo_coeff, mo_occ=mo_occ)
        vhf = mf.get_veff(mol, dm, dm_last, vhf)
        e_tot = mf.energy_tot(dm, h1e, vhf)
        
        # Here Fock matrix is h1e + vhf, without DIIS.  Calling get_fock
        # instead of the statement "fock = h1e + vhf" because Fock matrix may
        # be modified in some methods.
        fock = mf.get_fock(h1e, s1e, vhf, dm)  # = h1e + vhf, no DIIS
        fock+=v_emb
        
        norm_gorb = numpy.linalg.norm(mf.get_grad(mo_coeff, mo_occ, fock))
        if not TIGHT_GRAD_CONV_TOL:
            norm_gorb = norm_gorb / numpy.sqrt(norm_gorb.size)
        norm_ddm = numpy.linalg.norm(dm-dm_last)
        logger.info(mf, 'cycle= %d E= %.15g  delta_E= %4.3g  |g|= %4.3g  |ddm|= %4.3g',
                    cycle+1, e_tot, e_tot-last_hf_e, norm_gorb, norm_ddm)

        if callable(mf.check_convergence):
            scf_conv = mf.check_convergence(locals())
        elif abs(e_tot-last_hf_e) < conv_tol and norm_gorb < conv_tol_grad:
            scf_conv = True

        if dump_chk:
            mf.dump_chk(locals())

        if callable(callback):
            callback(locals())

#         cput1 = logger.timer(mf, 'cycle= %d'%(cycle+1), *cput1)

        if scf_conv:
            break

    if scf_conv and conv_check:
        # An extra diagonalization, to remove level shift
        #fock = mf.get_fock(h1e, s1e, vhf, dm)  # = h1e + vhf
        mo_energy, mo_coeff = mf.eig(fock, s1e)
        mo_occ = mf.get_occ(mo_energy, mo_coeff)
        dm, dm_last = mf.make_rdm1(mo_coeff, mo_occ), dm
        dm = lib.tag_array(dm, mo_coeff=mo_coeff, mo_occ=mo_occ)
        vhf = mf.get_veff(mol, dm, dm_last, vhf)
        e_tot, last_hf_e = mf.energy_tot(dm, h1e, vhf), e_tot

        fock = mf.get_fock(h1e, s1e, vhf, dm) + v_emb
        
        
        norm_gorb = numpy.linalg.norm(mf.get_grad(mo_coeff, mo_occ, fock))
        if not TIGHT_GRAD_CONV_TOL:
            norm_gorb = norm_gorb / numpy.sqrt(norm_gorb.size)
        norm_ddm = numpy.linalg.norm(dm-dm_last)

        conv_tol = conv_tol * 10
        conv_tol_grad = conv_tol_grad * 3
        if callable(mf.check_convergence):
            scf_conv = mf.check_convergence(locals())
        elif abs(e_tot-last_hf_e) < conv_tol or norm_gorb < conv_tol_grad:
            scf_conv = True
        logger.info(mf, 'Extra cycle  E= %.15g  delta_E= %4.3g  |g|= %4.3g  |ddm|= %4.3g',
                    e_tot, e_tot-last_hf_e, norm_gorb, norm_ddm)
        if dump_chk:
            mf.dump_chk(locals())

#     logger.timer(mf, 'scf_cycle', *cput0)
    # A post-processing hook before return
    mf.post_kernel(locals())
    return scf_conv, e_tot, mo_energy, mo_coeff, mo_occ


In [570]:
# define HFock obj (with embedded no. of e-)

full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED_HF) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6

In [571]:
# Find change of basis for localized orbs
# (NOTE Fock matrix is different with HFock compared to DFT ...
# hence why we recalc it)

S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_mol_EMBEDDED_HF.get_ovlp() , 0.5)

ortho_std = S_half@ full_system_scf.mo_coeff
ortho_loc = S_half@ C_all_localized_and_virt

# ortho_loc[:,0].dot(ortho_loc[:,2])

U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer

print(f'is U*C_ortho_loc =  C_ortho_STD: {(np.allclose(U @ ortho_loc, ortho_std))}')
print(f'is U Udag = Identity: {np.allclose(U.conj().T@U, np.eye(U.shape[0]))}')
                                          
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

V_chg_B_HFock = S_neg_half @ U @ S_half


print('V @ C_loc = C_std:', np.allclose(V_chg_B_HFock@C_all_localized_and_virt,
                                            full_system_scf.mo_coeff))

converged SCF energy = -69.6107725130686
is U*C_ortho_loc =  C_ortho_STD: True
is U Udag = Identity: True
V @ C_loc = C_std: True


In [572]:
# ############## include change of basis!
Hcore_std = EMBEDDED_full_system_scf_HF.get_hcore()
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V_chg_B_HFock)


EMBEDDED_full_system_scf_HF.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
                                                                                    EMBEDDED_full_system_scf_HF, #<-need this here for more expensive xc fucnctional@
                                                                                       V_chg_B_HFock,
                                                                                        dm=dm)

In [573]:

            
EMBEDDED_full_system_scf_HF.kernel = lambda mf, dm_env, dm_active, pyscf_DFT, PROJ_ortho,  conv_tol=1e-10, conv_tol_grad=None, dump_chk=True, dm0=None, callback=None, conv_check=True, **kwargs: kernel_NEW(
    EMBEDDED_full_system_scf_HF,
               dm_env,
                dm_active,
              pyscf_DFT,
            PROJ_ortho,
               conv_tol=1e-10,
               conv_tol_grad=None,
               dump_chk=True,
                 dm0=None,
                 callback=None, 
                 conv_check=True,
                 **kwargs)

In [575]:
EMBEDDED_full_system_scf_HF.max_cycle = 200

scf_conv, e_tot, mo_energy, mo_coeff, mo_occ = EMBEDDED_full_system_scf_HF.kernel(
    EMBEDDED_full_system_scf_HF,
    dm_enviro,
    dm_active, 
    test_full_system_scf, 
    PROJ_ortho) 


[-21.94623349   2.0436437    1.66067334  -0.40877309   0.04516006
   1.27584773   0.95616438]

[-22.20088816  -0.56895382  -0.71548364   0.97979784   0.75715911
   2.07212253   2.15876605]

[-22.2101587   -0.72931643  -0.61151824   0.82493425   0.93774413
   2.10817663   2.1725701 ]

[-22.27420097  -0.77155311  -0.53619546   0.7318902    0.94051781
   2.09923463   2.16613555]

[-22.26957574  -0.76950812  -0.44065233   0.78102314   0.9526441
   2.09964966   2.18051865]

[-22.27203904  -0.77115227  -0.44549259   0.77611985   0.95223506
   2.09921157   2.1806037 ]

[-22.21894573  -0.74173995  -0.36021969   0.86221799   0.9714689
   2.10390571   2.20147058]

[-22.27153741  -0.77478454  -0.40271096   0.75454317   0.95600807
   2.09762485   2.17810023]

[-22.19423553  -0.72365164  -0.54013606   0.78885967   0.95697204
   2.1068588    2.1693    ]

[-22.19022051  -0.70934009  -0.66345354   0.81303859   0.9400591
   2.11309679   2.16620079]

[-22.21497652  -0.72952934  -0.51495772   0.85309679 

[-22.20344332  -0.71039081  -0.63705638   0.78126191   0.93573683
   2.11869851   2.17818268]

[-22.20249871  -0.70517118  -0.63737854   0.78159582   0.93266486
   2.11595658   2.17803005]

[-22.18619524  -0.70692319  -0.6503324    0.79452568   0.93752998
   2.11481597   2.17536256]

[-22.19252707  -0.73774446  -0.64923124   0.81735022   0.95103821
   2.12090497   2.17384043]

[-22.20060897  -0.69903442  -0.64317528   0.84364345   0.92518734
   2.10921471   2.17564765]

[-22.18569859  -0.71592166  -0.64622701   0.83470333   0.93680573
   2.11428813   2.17477459]

[-22.19956929  -0.71568869  -0.63197525   0.83640758   0.93561614
   2.10913321   2.17687792]

[-22.21549686  -0.73849032  -0.62475549   0.83080879   0.94352696
   2.1148774    2.17729634]

[-22.2184132   -0.78635179  -0.62385127   0.81763716   0.96045642
   2.14816616   2.17875333]

[-22.27125638  -0.77747358  -0.62670918   0.80678305   0.92040916
   2.13453327   2.17298191]

[-22.24652127  -0.7622748   -0.65078099   0.818282

[-22.21417265  -0.72944703  -0.63257331   0.79884164   0.93699925
   2.1185849    2.17560498]

[-22.21099662  -0.71747098  -0.58631392   0.79697774   0.95244479
   2.10832737   2.18196207]

[-22.25093696  -0.87690189  -0.54933791   0.79658297   0.99836113
   2.12187322   2.18657251]

[-22.22002163  -0.76894489  -0.5294226    0.82422087   0.94101172
   2.11541398   2.18267857]

[-22.20637811  -0.71074856  -0.53799067   0.82975734   0.94355152
   2.10837122   2.18616661]

[-22.20264197  -0.73692241  -0.52122299   0.81873859   0.94977918
   2.11603376   2.18718809]

[-22.20182554  -0.73755459  -0.5360005    0.82489288   0.95113492
   2.12340998   2.18649343]

[-22.19340402  -0.68421639  -0.54399136   0.83183198   0.93244601
   2.11239154   2.1855928 ]

[-22.19035391  -0.75419822  -0.53022252   0.84483954   0.95619912
   2.12118261   2.18129353]

[-22.15873576  -0.68014457  -0.57472218   0.84665836   0.94853974
   2.10741147   2.18077348]

[-22.17889104  -0.71028335  -0.57962653   0.816130

In [576]:
2*len(active_MO_inds) == sum(mo_occ)

True

In [581]:
print(scf_conv)
print(e_tot)
mo_energy

False
-66.99829380651367


array([-22.20463498,  -0.75765242,  -0.59214652,   0.85289081,
         0.96415242,   2.11817433,   2.18467973])

In [ ]:
## manual projector (in HFock method - different to DFT definition!)

## 1. convert to orthogonal C_matrix
S_mat = EMBEDDED_full_system_scf_HF.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(S_mat , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(S_mat , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt # orthogonal C matrix (localized)

## 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
##### (do this in orthongoal C_matrix!)
### not we only take MO environment indices!
PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue

print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, enviro_MO_inds], 
        Loc_Ortho[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Is subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!


# undo ortho part!
PROJ_bas = S_neg_half@ PROJ_ortho  @ S_half

print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_bas@C_all_localized_and_virt[:, enviro_MO_inds], 
        C_all_localized_and_virt[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Is subsystem A traced out?: {
        np.allclose(PROJ_bas@C_all_localized_and_virt[:, active_MO_inds], 
        np.zeros_like(C_all_localized_and_virt[:, active_MO_inds]))}''') # # projected onto zeros!


In [ ]:
def Get_V_emb_huzinaga(scf_obj, PROJ_ortho, dm_env, dm_act, S_neg_half, S_half, Hcore_rotated):
    
    if scf_obj.mo_coeff is not None:
        print('HELLO')
        dm_act_CHANGING = scf_obj.make_rdm1(mo_coeff, pyscf_obj.mo_occ)
    else:
        print('FIRST')
        dm_act_CHANGING = dm_act
#         dm_act_CHANGING= scf_obj.init_guess_by_atom() #scf_obj.init_guess_by_1e()
#         dm_act_CHANGING= scf_obj.init_guess_by_1e()
    
    
    Fock = Hcore_rotated + scf_obj.get_veff(dm=dm_act_CHANGING + dm_env)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    projector = S_half @ projector_ortho  @ S_half
    
    g_A_and_B = scf_obj.get_veff(dm=dm_act_CHANGING+dm_env)
    g_A = scf_obj.get_veff(dm=dm_act_CHANGING)

    v_emb = g_A_and_B - g_A + projector
    
    if scf_obj.mo_coeff is not None:
        fock_orb_energy = scf_obj.mo_coeff.conj().T @(Fock+projector)@ scf_obj.mo_coeff
        print(fock_orb_energy)
    
    return v_emb
        
#### 3. NEXT USING PROJECTOR in ortho basis define projector via different methods
Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()
v_emb_HFOCK = Get_V_emb_huzinaga(EMBEDDED_full_system_scf_HF,
                                 PROJ_ortho,
                                 dm_enviro,
                                 dm_active,
                                 S_neg_half,
                                 S_half,
                                 Hcore_rotated)

projector = v_emb_HFOCK - EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active+dm_enviro) + EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active)

##########
# Fock_loc_basis = EMBEDDED_full_system_scf_HF.get_fock(dm=dm_active+dm_enviro)
Fock_loc_basis = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active+dm_enviro)


## projector should give opposite sign val of subsytem B orbitals vs standard Fock eval
proj_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @projector@C_all_localized_and_virt[:, enviro_MO_inds]
fock_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @Fock_loc_basis@C_all_localized_and_virt[:, enviro_MO_inds]

print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                          fock_orb_energy)} \n''')

new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)

## expect subsystem A orbital energies to remain the SAME
print('\n')
print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                    full_system_scf.mo_energy[active_MO_inds])} ''')

print('actually NOW that we are using HF Fock matix this should NO LONGER MATCH DFT orb energies!!! \n')

print(f'''enviro subsystem orbital energies should be ZERO (traced out):{ 
                               np.allclose(new_orbital_energies[enviro_MO_inds],
                                            np.zeros_like(new_orbital_energies[enviro_MO_inds]))} \n''')
    




print('\nbasically want active MOs first... then enviroment')
print(f'active MO indices: {active_MO_inds} \n')
print('should appear first when orbital inds listed by energy:')
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

##########



In [ ]:
bsdbsdfb

In [ ]:
Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()
## overwrite h_core to include embedding term!!!!
# EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb_HFOCK + Hcore_rotated
EMBEDDED_full_system_scf_HF.get_hcore = lambda  *args: Get_V_emb_huzinaga(EMBEDDED_full_system_scf_HF,
                                                                         PROJ_ortho,
                                                                         dm_enviro,
                                                                         dm_active,
                                                                         S_neg_half,
                                                                         S_half,
                                                                         Hcore_rotated) + Hcore_rotated

##############
EMBEDDED_full_system_scf_HF.kernel() 



In [ ]:
#alg


# guess Dmat
# H_core = EMBEDDED_full_system_scf_HF.get_hcore()
# D = EMBEDDED_full_system_scf_HF.init_guess_by_1e()
D = dm_active
# mo_coeff= C_active
projector = S_neg_half@ PROJ_ortho  @ S_half


max_iter = 100
HF_energy =0
E_previous = 0
E_tol = 1e-6


H_core_normal = EMBEDDED_full_system_scf_HF.get_hcore()
H_core_MODIFIED = H_core_normal + g_A_and_B - g_A + projector

for i in range(max_iter+1):
    
    # Build Fock Matrix
    F_A = H_core_MODIFIED +  EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    
    Huz_op = -0.5 * (F_A@projector + projector@F_A)
    
    vhf = EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    
    F_A = H_core_MODIFIED + vhf
    
    ### find RHF energy
    e1 = np.einsum('ij,ji->', H_core_MODIFIED, D)
    e_coul = np.einsum('ij,ji->', vhf, D) * .5
    HF_energy = e1 + e_coul + EMBEDDED_full_system_scf_HF.mol.energy_nuc()
#     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
    
    
    mo_energy, mo_coeff = EMBEDDED_full_system_scf_HF.eig(F_A, S_mat)
    mo_occ = EMBEDDED_full_system_scf_HF.get_occ(mo_energy, mo_coeff)
    D = EMBEDDED_full_system_scf_HF.make_rdm1(mo_coeff=mo_coeff, mo_occ=mo_occ)
    
    ###
    # supermolecular Fock matrix
    Fock = Hcore_rotated + EMBEDDED_full_system_scf_HF.get_veff(dm=D + dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    projector = S_half @ projector_ortho  @ S_half
    
    Fock_emb = H_core_normal + g_A_and_B - g_A + projector + EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    proj_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @projector@mo_coeff[:, enviro_MO_inds]
    fock_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @Fock_emb@mo_coeff[:, enviro_MO_inds]

    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    ###
    
    # if not convereged store old result
    E_previous = HF_energy
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy} iteration: {i}')

In [ ]:
#alg


# guess Dmat
# H_core = EMBEDDED_full_system_scf_HF.get_hcore()
# D = EMBEDDED_full_system_scf_HF.init_guess_by_1e()
D = dm_active
# mo_coeff= C_active
projector = S_neg_half@ PROJ_ortho  @ S_half


max_iter = 100
HF_energy =0
E_previous = 0
E_tol = 1e-6


H_core_normal = EMBEDDED_full_system_scf_HF.get_hcore()
for i in range(max_iter+1):
    
    # Build Fock Matrix
#     H_core_MODIFIED = H_core_normal + g_A_and_B - g_A + projector
#     F_A = H_core_MODIFIED + EMBEDDED_full_system_scf_HF.get_veff(dm=D + dm_enviro)

    H_core_MODIFIED = H_core_normal + g_A_and_B - g_A + projector
    vhf = EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    
    F_A = H_core_MODIFIED + vhf
    
    ### find RHF energy
    e1 = np.einsum('ij,ji->', H_core_MODIFIED, D)
    e_coul = np.einsum('ij,ji->', vhf, D) * .5
    HF_energy = e1 + e_coul + EMBEDDED_full_system_scf_HF.mol.energy_nuc()
#     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
    
    
    mo_energy, mo_coeff = EMBEDDED_full_system_scf_HF.eig(F_A, S_mat)
    mo_occ = EMBEDDED_full_system_scf_HF.get_occ(mo_energy, mo_coeff)
    D = EMBEDDED_full_system_scf_HF.make_rdm1(mo_coeff=mo_coeff, mo_occ=mo_occ)
    
    ###
    # supermolecular Fock matrix
    Fock = Hcore_rotated + EMBEDDED_full_system_scf_HF.get_veff(dm=D + dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    projector = S_half @ projector_ortho  @ S_half
    
    Fock_emb = H_core_normal + g_A_and_B - g_A + projector + EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    proj_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @projector@mo_coeff[:, enviro_MO_inds]
    fock_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @Fock_emb@mo_coeff[:, enviro_MO_inds]

    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    ###
    
    # if not convereged store old result
    E_previous = HF_energy
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy} iteration: {i}')

In [ ]:
np.trace(S_mat@(D + dm_enviro))

In [ ]:
print(proj_orb_energy)
print(fock_orb_energy)

In [ ]:
#alg


# guess Dmat
# H_core = EMBEDDED_full_system_scf_HF.get_hcore()
# D = EMBEDDED_full_system_scf_HF.init_guess_by_1e()
D = dm_active
mo_coeff = C_active #C_active + C_envrio

max_iter = 100
HF_energy =0
E_previous = 0
E_tol = 1e-6


H_core_normal = EMBEDDED_full_system_scf_HF.get_hcore()
for i in range(max_iter+1):
    
    # Build Fock Matrix
    C_new = mo_coeff
    C_new[:, enviro_MO_inds] +=C_envrio
    Loc_Ortho = S_half@ C_new 

    PROJ_ortho = np.zeros_like(S_mat)
    for MO_ind in range(Loc_Ortho.shape[1]):
        if MO_ind in enviro_MO_inds:
            outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
            PROJ_ortho+=outer
    
    Fock = H_core_normal + EMBEDDED_full_system_scf_HF.get_veff(dm=D+dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    projector = S_half @ projector_ortho  @ S_half
    
    v_emb = g_A_and_B - g_A + projector
    H_core_MODIFIED = v_emb + H_core_normal
    
    vhf = EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    
    ### find RHF energy
    e1 = np.einsum('ij,ji->', H_core_MODIFIED, D)
    e_coul = np.einsum('ij,ji->', vhf, D) * .5
    HF_energy = e1 + e_coul + EMBEDDED_full_system_scf_HF.mol.energy_nuc()
#     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
    
    
#     fock =  H_core_MODIFIED + EMBEDDED_full_system_scf_HF.get_veff(dm=D)
#     fock = EMBEDDED_full_system_scf_HF.get_fock(H_core_MODIFIED, S_mat,  EMBEDDED_full_system_scf_HF.get_veff(dm=D), D)  # = h1e + vhf, no DIIS
    Fock_emb = H_core_MODIFIED + vhf
    
    mo_energy, mo_coeff = EMBEDDED_full_system_scf_HF.eig(Fock_emb, S_mat)
    mo_occ = EMBEDDED_full_system_scf_HF.get_occ(mo_energy, mo_coeff)
    D = EMBEDDED_full_system_scf_HF.make_rdm1(mo_coeff=mo_coeff, mo_occ=mo_occ)
    
    ###
    proj_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @projector@mo_coeff[:, enviro_MO_inds]
    fock_orb_energy = mo_coeff[:, enviro_MO_inds].conj().T @fock@mo_coeff[:, enviro_MO_inds]

    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    ###
    
    # if not convereged store old result
    E_previous = HF_energy
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy} iteration: {i}')

In [ ]:
proj_orb_energy = mo_coeff.conj().T @projector@mo_coeff
fock_orb_energy = mo_coeff.conj().T @fock@mo_coeff

print(np.diag(proj_orb_energy))
print(np.diag(fock_orb_energy))

np.diag(mo_coeff.conj().T @(fock+projector)@mo_coeff)

In [ ]:
Fock =  EMBEDDED_full_system_scf_HF.get_hcore()  + EMBEDDED_full_system_scf_HF.get_veff(dm=D+dm_enviro)
np.diag(mo_coeff.conj().T @ (Fock+projector) @ mo_coeff)


In [ ]:
# #alg

# S = full_system_mol.intor('int1e_ovlp') # 1e- electron overlap <i|j>
# eri_ao = full_system_mol.intor('int2e') # 2e- electron repulsion integrals in AO basis <ij|kl>
# U = sp.linalg.fractional_matrix_power(S, -0.5)

# Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()


# ### GUESS
# F_guess = Hcore_rotated
# F_prime = U.conj().T @ F_guess @ U
# epsilon, C_prime = np.linalg.eigh(F_prime)
# # transofrm C_prime back to AO basis
# C = U @ C_prime
# # get occupied orbitals
# n_docc = 2*len(active_MO_inds)

# C_occ = C[:, :n_docc]
# # build density matrix from occupied orbitals!
# D = C_occ @ C_occ.T


# ## run HFock
# max_iter = 100
# HF_energy =0
# E_previous = 0
# E_tol = 1e-6

# for i in range(max_iter+1):

#     # Build Fock Matrix
# #     J_mat = np.einsum('mvls,ls -> mv', eri_ao, D, optimize=True)
# #     K_mat = np.einsum('mlvs,ls -> mv', eri_ao, D, optimize=True)
# #     Fock = H_core + 2*J_mat - K_mat
    
#     Fock = Hcore_rotated + EMBEDDED_full_system_scf_HF.get_veff(dm=D + dm_enviro)
#     Fock_ortho = S_neg_half@Fock @ S_neg_half
#     projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
#     projector = S_half @ projector_ortho  @ S_half
    
#     v_emb = g_A_and_B - g_A + projector
    
#     Fock =  Fock + v_emb
    
    
    
#     ### find RHF energy
#     HF_energy = np.einsum('pq,pq ->', D, (Fock+Hcore_rotated), optimize=True) + full_system_mol.energy_nuc()
# #     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
#     # (no 0.5 here as included in D mat natively!)
    
#     ### check convergence
#     if np.abs(HF_energy-E_previous)<E_tol:
#         break
        
        
#     # if not convereged store old result
#     E_previous = HF_energy
    
#     ## compute new orbital guess
#     F_prime = U.conj().T @ Fock @ U

#     epsilon, C_prime = np.linalg.eigh(F_prime)

#     # transofrm C_prime back to AO basis
#     C = U @ C_prime

#     # get occupied orbitals
#     C_occ = C[:, :n_docc]

#     # build density matrix from occupied orbitals!
# #     D = C_occ @ C_occ.T
#     D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    
#     if i==max_iter:
#         raise ValueError('Maximum number of SCF iterations exceeded')
        
# print(f'final RHF SCF energy {HF_energy} iteration: {i}')

In [ ]:
print(EMBEDDED_full_system_scf_HF.mo_energy)
print()
print(EMBEDDED_full_system_scf_HF.mo_occ)

C_active_embedd_opt = EMBEDDED_full_system_scf_HF.mo_coeff[:,EMBEDDED_full_system_scf_HF.mo_occ>0]
HF_dm_mat = 2* C_active_embedd_opt @ C_active_embedd_opt.conj().T

In [ ]:
# emb_orbs = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              EMBEDDED_full_system_scf_HF.mo_coeff, 
#                              list(range(EMBEDDED_full_system_scf_HF.mo_coeff.shape[1])),
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [ ]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

if projector_method == 'mu_shfit':
    embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
elif projector_method == 'huzinaga':
#     embedded_cc_obj.frozen  = enviro_MO_inds.tolist()
    embedded_cc_obj.frozen  = list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))
else:
    raise ValueError(f'unknown projector method {projector_method}')


# embedded_cc_obj.frozen = None
e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=HF_dm_mat),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf


In [ ]:
embedded_cc_obj.frozen

In [ ]:
WF_correction = np.einsum('ij, ij', v_emb, dm_active)

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 


In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

In [ ]:
Fock_emb = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=HF_dm_mat)


print(EMBEDDED_full_system_scf_HF.mo_energy)
print()
orbE = C_active_embedd_opt.conj().T @ Fock_emb @ C_active_embedd_opt
print(np.around(orbE, 7))

np.allclose(EMBEDDED_full_system_scf_HF.mo_energy[:len(active_MO_inds)],
           np.diag(orbE))


In [ ]:
gsdfgsdf

This contains the DFT embedding term (in Veff of Hartree Fock calc)

Whereas in above calc Veff contains v_emb_FOCK (defined using HFock theory!)

DON'T KNOW WHICH ONE IS CORRECT

In [ ]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6


############## include change of basis!
Hcore_std = EMBEDDED_full_system_scf_HF.get_hcore()
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


EMBEDDED_full_system_scf_HF.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
                                                                                    EMBEDDED_full_system_scf_HF, #<-need this here for more expensive xc fucnctional@
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()

## overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb + Hcore_rotated

##############



EMBEDDED_full_system_scf_HF.kernel() 


# # # overwrite orbs with RKS embedded orbs!
# EMBEDDED_full_system_scf_HF.mo_coeff = full_system_scf_HIGH_LEVEL.mo_coeff
# EMBEDDED_full_system_scf_HF.mo_occ = full_system_scf_HIGH_LEVEL.mo_occ 
# EMBEDDED_full_system_scf_HF.mo_energy = full_system_scf_HIGH_LEVEL.mo_energy

# print(EMBEDDED_full_system_scf_HF.energy_elec(dm=dm_active_EMBEDDED_high_lvl)[0])
# print(EMBEDDED_full_system_scf_HF.energy_tot(dm=dm_active_EMBEDDED_high_lvl))
# print()
# print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
# print(EMBEDDED_full_system_scf_HF.energy_tot())

In [ ]:
EMBEDDED_full_system_scf_HF.mo_energy

In [ ]:
# full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
#                       basis=basis,
#                        charge=charge,
#                        spin=spin,
#                       )
# full_system_mol_EMBEDDED_HF.build()

# # RE-DEFINE number of electrons in system
# full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

# EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
# EMBEDDED_full_system_scf_HF.verbose=1
# EMBEDDED_full_system_scf_HF.max_memory= memory
# EMBEDDED_full_system_scf_HF.conv_tol = 1e-6


# ############## include change of basis!
# Hcore_std = EMBEDDED_full_system_scf_HF.get_hcore()
# EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
#                                                              Unitary_rot=V)

# # mistake here with HIGH_LEVEL TERM
# # MISTAKE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! (NEED HFOCK OBJECT NOT DFT)
# full_system_scf_HIGH_LEVEL.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
#                                                                                     EMBEDDED_full_system_scf_HF, #<-need this here for more expensive xc fucnctional@
#                                                                                        V,
#                                                                                         dm=dm)

# Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()

# ## overwrite h_core to include embedding term!!!!
# EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb + Hcore_rotated

# ##############



# EMBEDDED_full_system_scf_HF.kernel() # <------ do NOT RUN!


# # # # overwrite orbs with RKS embedded orbs!
# # EMBEDDED_full_system_scf_HF.mo_coeff = full_system_scf_HIGH_LEVEL.mo_coeff
# # EMBEDDED_full_system_scf_HF.mo_occ = full_system_scf_HIGH_LEVEL.mo_occ 
# # EMBEDDED_full_system_scf_HF.mo_energy = full_system_scf_HIGH_LEVEL.mo_energy

# # print(EMBEDDED_full_system_scf_HF.energy_elec(dm=dm_active_EMBEDDED_high_lvl)[0])
# # print(EMBEDDED_full_system_scf_HF.energy_tot(dm=dm_active_EMBEDDED_high_lvl))
# # print()
# # print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
# # print(EMBEDDED_full_system_scf_HF.energy_tot())

In [ ]:
print(EMBEDDED_full_system_scf_HF.mo_energy)
print(EMBEDDED_full_system_scf_HF.mo_occ)

C_active_embedd_opt = EMBEDDED_full_system_scf_HF.mo_coeff[:,EMBEDDED_full_system_scf_HF.mo_occ>0]
HF_dm_mat = 2* C_active_embedd_opt @ C_active_embedd_opt.conj().T

In [ ]:
emb_Fock = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=HF_dm_mat)


print(EMBEDDED_full_system_scf_HF.mo_energy)
print()
orbE = C_active_embedd_opt.conj().T @ emb_Fock @ C_active_embedd_opt
print(np.around(orbE, 7))

np.allclose(EMBEDDED_full_system_scf_HF.mo_energy[:len(active_MO_inds)],
           np.diag(orbE))


In [ ]:
# # TODO:

# May have to keep re-defining Huzinaga operator to get rid of those orbitals 
# As I think one iteration of HFock undoes what the projector operator is doing
# Need to check this though (as clearly Huz is defiend by DFT enviroment NOT the optimized one)
# so it could actually be okay!

# Just it is clear from mu shift orbs what is the environement
# not sure if for Huz op the env orbs should be at zero energy! 

In [ ]:
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

In [ ]:
enviro_MO_inds

In [ ]:
## choose frozen from here^^^^
print(enviro_MO_inds)
print(f'for mu shift expect it to be the LAST {len(enviro_MO_inds)}')

print(f'''for Huzinaga expect it to be positions {list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))}''')


In [ ]:
# TODO: Choice of frozen orbs for Huzinaga method could be wrong
# need to CHECK this!

In [ ]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

if projector_method == 'mu_shfit':
    embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
elif projector_method == 'huzinaga':
    embedded_cc_obj.frozen  = enviro_MO_inds.tolist()
#     embedded_cc_obj.frozen  = list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))
else:
    raise ValueError(f'unknown projector method {projector_method}')


e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=HF_dm_mat),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf


In [ ]:
projector_method

In [ ]:
# print('huzinaga HFock = ', -16.203577904621056)
# print('mu_shfit HFock = ', -16.028522482287855)

In [ ]:
WF_correction = np.einsum('ij, ij', v_emb, dm_active)

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 

In [ ]:

print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

In [ ]:
fkaisdjlfkds

# QC part

In [ ]:
N_enviroment_MOs = len(enviro_MO_inds) 

one_body_integrals, two_body_integrals = vqe_in_dft.Get_embedded_one_and_two_body_integrals_MO_basis(EMBEDDED_full_system_scf_HF,
                                                                                            N_enviroment_MOs,
                                                                                            physists_notation=phys_notation)


print(one_body_integrals.shape)
print(two_body_integrals.shape)

In [ ]:
print(active_MO_inds)
print(enviro_MO_inds)

In [ ]:
one_body_terms, two_body_terms = vqe_in_dft.Get_SpinOrbs_from_Spatial(one_body_integrals,
                                                                   two_body_integrals,
                                                                   physists_notation=phys_notation,
                                                                   EQ_Tolerance=1e-8)


In [ ]:
Nuclear_energy =  full_system_scf.energy_nuc()

H_fermionic = vqe_in_dft.Get_fermionic_H(one_body_terms, 
                                     two_body_terms, 
                                     Nuclear_energy,
                                     core_constant=0, 
                                     physists_notation=phys_notation,
                                        jupyter_notebook=True)



len(list(H_fermionic))

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(H_fermionic)




eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_sparse, which='SA', k=1)

In [ ]:
WF_correction  = np.einsum('ij, ij', v_emb, dm_active) # note different definition

E_VQE = eigvals_EMBED[0]  + E_env + two_e_cross - WF_correction
E_VQE 

In [ ]:
N_electrons_expected = 2*len(active_MO_inds)
N_electrons_Q_state = np.binary_repr(np.where(np.abs(eigvecs_EMBED)>1e-2)[0][0]).count('1') 

print(f'expect {N_electrons_expected} electrons')
print(f'quantum state has {N_electrons_Q_state} electrons \n')

print(f'number of electrons correct: {N_electrons_expected == N_electrons_Q_state}')

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))
print('VQE error:', np.abs(E_VQE-my_fci.e_tot))